In [5]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import requests
import time
from shapely.geometry import box

# --- 1. Baixar o polígono do arthur alvim ---
bairro = ox.geocode_to_gdf("Tatuape, São Paulo")

# --- 2. Criar a grade (grid) sobre o bairro ---
def criar_grid(bairro, tamanho=0.005):  # Aproximadamente 200m
    minx, miny, maxx, maxy = bairro.total_bounds
    grid = []
    x = minx
    while x < maxx:
        y = miny
        while y < maxy:
            grid.append(box(x, y, x + tamanho, y + tamanho))
            y += tamanho
        x += tamanho
    grid_gdf = gpd.GeoDataFrame(geometry=grid, crs=bairro.crs)
    grid_gdf = gpd.overlay(grid_gdf, bairro, how='intersection')
    return grid_gdf

grid = criar_grid(bairro)


# --- 3. Corrigir projeção para metros, calcular centroide ---
grid = grid.to_crs(epsg=3857)  # Web Mercator
grid["centroide"] = grid.geometry.centroid

# --- 4. Criar colunas separadas de latitude/longitude
centroides = grid["centroide"].to_crs(epsg=4326)  # voltar para lat/lon
grid["centroide_lon"] = centroides.geometry.x
grid["centroide_lat"] = centroides.geometry.y

# --- 6. Definir hospital e API ---
API_KEY = "5b3ce3597851110001cf624824d680a2a89c4dc48933794aaca7b81d"  # Troque aqui pela sua chave
hospital_coords = [-46.5663866, -23.5331239] 

def calcular_distancia_km(origem, destino, api_key):
    url = "https://api.openrouteservice.org/v2/directions/driving-car/json"
    headers = {
        "Authorization": api_key,
        "Content-Type": "application/json"
    }
    body = {
        "coordinates": [origem, destino]
    }
    try:
        response = requests.post(url, json=body, headers=headers)
        if response.status_code == 200:
            dados = response.json()
            distancia_metros = dados['routes'][0]['summary']['distance']
            distancia_km = distancia_metros / 1000
            return round(distancia_km, 2)
        else:
            print("Erro na API:", response.text)
            return None
    except Exception as e:
        print(f"Erro: {e}")
        return None

# --- 7. Calcular a distância para cada grid ---
distancias_km = []
for i, row in grid.iterrows():
    origem = [row["centroide_lon"], row["centroide_lat"]]
    distancia = calcular_distancia_km(origem, hospital_coords, API_KEY)
    distancias_km.append(distancia)
    time.sleep(1)  # evita limite da API gratuita

grid["distancia_km"] = distancias_km
grid.to_csv(f"estimativa_tatuape.csv", index=False)

In [6]:
grid

,bbox_west,bbox_south,bbox_east,bbox_north,place_id,osm_type,osm_id,lat,lon,class,...,place_rank,importance,addresstype,name,display_name,geometry,centroide,centroide_lon,centroide_lat,distancia_km
0,-46.585013,-23.556088,-46.548245,-23.522932,8362914,relation,3049455,-23.540891,-46.575009,boundary,...,18,0.379786,suburb,Tatuapé,"Tatuapé, São Paulo, Região Imediata de São Pau...","POLYGON ((-5185263.328 -2697586.561, -5185263....",POINT (-5185290.566 -2697713.14),-46.580258,-23.542130,2.49
1,-46.585013,-23.556088,-46.548245,-23.522932,8362914,relation,3049455,-23.540891,-46.575009,boundary,...,18,0.379786,suburb,Tatuapé,"Tatuapé, São Paulo, Região Imediata de São Pau...","POLYGON ((-5185263.328 -2696979.446, -5185263....",POINT (-5185370.535 -2697220.866),-46.580976,-23.538076,2.15
2,-46.585013,-23.556088,-46.548245,-23.522932,8362914,relation,3049455,-23.540891,-46.575009,boundary,...,18,0.379786,suburb,Tatuapé,"Tatuapé, São Paulo, Região Imediata de São Pau...","POLYGON ((-5185263.328 -2696372.355, -5185263....",POINT (-5185444.992 -2696656.269),-46.581645,-23.533426,2.17
3,-46.585013,-23.556088,-46.548245,-23.522932,8362914,relation,3049455,-23.540891,-46.575009,boundary,...,18,0.379786,suburb,Tatuapé,"Tatuapé, São Paulo, Região Imediata de São Pau...","POLYGON ((-5185263.328 -2696372.355, -5185692....",POINT (-5185496.726 -2696153.465),-46.582110,-23.529285,2.22
4,-46.585013,-23.556088,-46.548245,-23.522932,8362914,relation,3049455,-23.540891,-46.575009,boundary,...,18,0.379786,suburb,Tatuapé,"Tatuapé, São Paulo, Região Imediata de São Pau...","POLYGON ((-5184706.731 -2698800.859, -5184706....",POINT (-5184842.507 -2698872.212),-46.576233,-23.551676,3.89
5,-46.585013,-23.556088,-46.548245,-23.522932,8362914,relation,3049455,-23.540891,-46.575009,boundary,...,18,0.379786,suburb,Tatuapé,"Tatuapé, São Paulo, Região Imediata de São Pau...","POLYGON ((-5184706.731 -2698193.699, -5184706....",POINT (-5184928.918 -2698477.301),-46.577009,-23.548424,3.60
6,-46.585013,-23.556088,-46.548245,-23.522932,8362914,relation,3049455,-23.540891,-46.575009,boundary,...,18,0.379786,suburb,Tatuapé,"Tatuapé, São Paulo, Região Imediata de São Pau...","POLYGON ((-5185263.328 -2697586.561, -5184706....",POINT (-5184981.185 -2697886.769),-46.577478,-23.543560,2.49
7,-46.585013,-23.556088,-46.548245,-23.522932,8362914,relation,3049455,-23.540891,-46.575009,boundary,...,18,0.379786,suburb,Tatuapé,"Tatuapé, São Paulo, Região Imediata de São Pau...","POLYGON ((-5185263.328 -2697586.561, -5185263....",POINT (-5184985.03 -2697283.004),-46.577513,-23.538588,1.72
8,-46.585013,-23.556088,-46.548245,-23.522932,8362914,relation,3049455,-23.540891,-46.575009,boundary,...,18,0.379786,suburb,Tatuapé,"Tatuapé, São Paulo, Região Imediata de São Pau...","POLYGON ((-5185263.328 -2696979.446, -5185263....",POINT (-5184985.03 -2696675.9),-46.577513,-23.533588,1.39
9,-46.585013,-23.556088,-46.548245,-23.522932,8362914,relation,3049455,-23.540891,-46.575009,boundary,...,18,0.379786,suburb,Tatuapé,"Tatuapé, São Paulo, Região Imediata de São Pau...","POLYGON ((-5184706.731 -2696372.355, -5185263....",POINT (-5184975.15 -2696091.539),-46.577424,-23.528775,2.08


In [8]:
%pip install folium
import folium
from shapely.geometry import mapping

# Converter o bairro para EPSG:4326 (lat/lon) para exibição no mapa
bairro_wgs84 = bairro.to_crs(epsg=4326)
grid_wgs84 = grid.to_crs(epsg=4326)

# Criar mapa centralizado no bairro
centroide_bairro = bairro_wgs84.geometry.centroid.iloc[0]
m = folium.Map(location=[centroide_bairro.y, centroide_bairro.x], zoom_start=14)

# Adicionar polígono do bairro
folium.GeoJson(bairro_wgs84.geometry.__geo_interface__,
               name="Bairro - Tatuape",
               style_function=lambda x: {"color": "black", "fillOpacity": 0.1}
).add_to(m)

# Adicionar os grids
for _, row in grid_wgs84.iterrows():
    folium.GeoJson(mapping(row.geometry),
                   style_function=lambda x: {"color": "blue", "weight": 1, "fillOpacity": 0}
    ).add_to(m)

# Adicionar ponto do hospital
, 
folium.Marker(
    location=[-23.5331239, -46.5663866] ,
    popup="Hospital Itaquera",
    icon=folium.Icon(color='red', icon='plus-sign')
).add_to(m)

# Mostrar mapa
m

# [-46.5315852, -23.5605378] 


[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip
C:\Users\joth1\AppData\Local\Temp\ipykernel_32920\4183604386.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroide_bairro = bairro_wgs84.geometry.centroid.iloc[0]


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: folium in c:\users\joth1\appdata\local\programs\python\python312\lib\site-packages (0.19.5)

